In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import RobustScaler 
from sklearn.model_selection import train_test_split

C:\Users\APC\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
train_2016 = pd.read_csv('./../preprocessed data/df_train_2016.csv')
predict_2016 = pd.read_csv('./../preprocessed data/df_predict_2016.csv')

train_2017 = pd.read_csv('./../preprocessed data/df_train_2017.csv')
predict_2017 = pd.read_csv('./../preprocessed data/df_predict_2017.csv')

sample = pd.read_csv("./../data/sample_submission.csv")

In [3]:
def standardize(df):
    numerical_columns = ['bathroomcnt', 'bedroomcnt','calculatedfinishedsquarefeet',
       'fireplacecnt', 'garagecarcnt', 'garagetotalsqft','latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum','unitcnt',
       'yardbuildingsqft17', 'structuretaxvaluedollarcnt',
       'landtaxvaluedollarcnt', 'age']
    for col in numerical_columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        Q2 = df[col].quantile(0.5)
        if Q3>Q1:
            df[col] = (df[col]-Q2)/(Q3-Q1)
    return df

In [4]:
def polynomial(df, most_corr_feat):
    for var in most_corr_feat:
        s2_var_name = var + '-s2'
        s3_var_name = var + '-s3'
        sq_var_name = var + '-sqrt'

        # Create features 
        df[s2_var_name] = df[var] ** 2 
        df[s3_var_name] = df[var] ** 3 
        df[sq_var_name] = np.sqrt(df[var]) 
    return df

In [5]:
def outliner_drop(df):
    Q1 = df['logerror'].quantile(0.25)
    Q3 = df['logerror'].quantile(0.75)
    IQR = Q3-Q1
    lower_lim = Q1 - 1.5*IQR
    upper_lim = Q3 + 1.5*IQR
    df=df[df.logerror >= lower_lim ]
    df=df[df.logerror <= upper_lim ]
    return df

# 2016 

In [6]:
numerical_columns = ['logerror','bathroomcnt', 'bedroomcnt','calculatedfinishedsquarefeet',
       'fireplacecnt', 'garagecarcnt', 'garagetotalsqft', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum','unitcnt',
       'yardbuildingsqft17', 'structuretaxvaluedollarcnt',
       'landtaxvaluedollarcnt', 'age']
corr = train_2016[numerical_columns].corr()
most_corr_feat = corr.logerror.abs().sort_values(ascending=False)[1:9].index
most_corr_feat

Index(['calculatedfinishedsquarefeet', 'bathroomcnt', 'bedroomcnt',
       'structuretaxvaluedollarcnt', 'age', 'poolcnt', 'garagetotalsqft',
       'fireplacecnt'],
      dtype='object')

In [7]:
train_16 = train_2016.copy()
train_16 = polynomial(train_16,most_corr_feat)
train_16 = standardize(train_16)
train_16 = outliner_drop(train_16)
y = train_16['logerror']
train_16 = train_16.drop(['logerror'],axis=1)

predict_16 = predict_2016.drop(['parcelid'],axis=1)
predict_16 = polynomial(predict_16,most_corr_feat)
predict_16 = standardize(predict_16)

In [8]:
# Set of alpha values to test
#alphas = np.logspace(7,10)
alphas = np.arange(28800000,28810000,100)
print(f"Testing with alphas={alphas}")

# Tune Ridge Regression
ridgecv = RidgeCV(alphas=alphas)
ridgecv.fit(train_16, y)
print(f"Best Ridge Alpha: {ridgecv.alpha_}")

Testing with alphas=[28800000 28800100 28800200 28800300 28800400 28800500 28800600 28800700
 28800800 28800900 28801000 28801100 28801200 28801300 28801400 28801500
 28801600 28801700 28801800 28801900 28802000 28802100 28802200 28802300
 28802400 28802500 28802600 28802700 28802800 28802900 28803000 28803100
 28803200 28803300 28803400 28803500 28803600 28803700 28803800 28803900
 28804000 28804100 28804200 28804300 28804400 28804500 28804600 28804700
 28804800 28804900 28805000 28805100 28805200 28805300 28805400 28805500
 28805600 28805700 28805800 28805900 28806000 28806100 28806200 28806300
 28806400 28806500 28806600 28806700 28806800 28806900 28807000 28807100
 28807200 28807300 28807400 28807500 28807600 28807700 28807800 28807900
 28808000 28808100 28808200 28808300 28808400 28808500 28808600 28808700
 28808800 28808900 28809000 28809100 28809200 28809300 28809400 28809500
 28809600 28809700 28809800 28809900]
Best Ridge Alpha: 28800000


In [9]:
rlr = Ridge(alpha=ridgecv.alpha_)
rlr.fit(train_16, y)

y_pred=rlr.predict(train_16)
df_y=abs(y-y_pred)
MAE=df_y.sum()/len(y)
print(MAE)

0.03544808577439068


C:\Users\APC\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=3.09851e-33): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


In [10]:
Y = rlr.predict(predict_16)
sample['201610'] = Y
sample['201611'] = Y
sample['201612'] = Y

# 2017

In [20]:
numerical_columns = ['logerror','bathroomcnt', 'bedroomcnt','calculatedfinishedsquarefeet',
       'fireplacecnt', 'garagecarcnt', 'garagetotalsqft', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum','unitcnt',
       'yardbuildingsqft17', 'structuretaxvaluedollarcnt',
       'landtaxvaluedollarcnt', 'age']
corr = train_2016[numerical_columns].corr()
most_corr_feat = corr.logerror.abs().sort_values(ascending=False)[1:9].index
most_corr_feat

Index(['calculatedfinishedsquarefeet', 'bathroomcnt', 'bedroomcnt',
       'structuretaxvaluedollarcnt', 'age', 'poolcnt', 'garagetotalsqft',
       'fireplacecnt'],
      dtype='object')

In [21]:
train_17 = train_2017.copy()
train_17 = polynomial(train_17,most_corr_feat)
train_17 = standardize(train_17)
train_17 = outliner_drop(train_17)
y = train_17['logerror']
train_17 = train_17.drop(['logerror'],axis=1)

predict_17 = predict_2017.drop(['parcelid'],axis=1)
predict_17 = polynomial(predict_17,most_corr_feat)
predict_17 = standardize(predict_17)

In [22]:
# Set of alpha values to test
alphas = np.logspace(8,11)
#alphas = np.arange(28800000,28810000,100)
print(f"Testing with alphas={alphas}")

# Tune Ridge Regression
ridgecv = RidgeCV(alphas=alphas)
ridgecv.fit(train_17, y)
print(f"Best Ridge Alpha: {ridgecv.alpha_}")

Testing with alphas=[1.00000000e+08 1.15139540e+08 1.32571137e+08 1.52641797e+08
 1.75751062e+08 2.02358965e+08 2.32995181e+08 2.68269580e+08
 3.08884360e+08 3.55648031e+08 4.09491506e+08 4.71486636e+08
 5.42867544e+08 6.25055193e+08 7.19685673e+08 8.28642773e+08
 9.54095476e+08 1.09854114e+09 1.26485522e+09 1.45634848e+09
 1.67683294e+09 1.93069773e+09 2.22299648e+09 2.55954792e+09
 2.94705170e+09 3.39322177e+09 3.90693994e+09 4.49843267e+09
 5.17947468e+09 5.96362332e+09 6.86648845e+09 7.90604321e+09
 9.10298178e+09 1.04811313e+10 1.20679264e+10 1.38949549e+10
 1.59985872e+10 1.84206997e+10 2.12095089e+10 2.44205309e+10
 2.81176870e+10 3.23745754e+10 3.72759372e+10 4.29193426e+10
 4.94171336e+10 5.68986603e+10 6.55128557e+10 7.54312006e+10
 8.68511374e+10 1.00000000e+11]
Best Ridge Alpha: 100000000.0


In [23]:
rlr = Ridge(alpha=ridgecv.alpha_)
rlr.fit(train_17, y)

y_pred=rlr.predict(train_17)
df_y=abs(y-y_pred)
MAE=df_y.sum()/len(y)
print(MAE)

0.034824034857191724


C:\Users\APC\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.21174e-32): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


In [15]:
Y = rlr.predict(predict_17)
sample['201610'] = Y
sample['201611'] = Y
sample['201612'] = Y

In [16]:
sample.to_csv('./../submission/ridge.csv', index=False, float_format='%.4f')